Haplogroup convergence
===
In human genetics, the haplogroups most commonly studied are Y-chromosome (Y-DNA) haplogroups and mitochondrial DNA (mtDNA) haplogroups, both of which can be used to define genetic populations. Y-DNA is passed solely along the patrilineal line.

Simply put, The haplo group is a property that is passed from father to son unchanged, with very few mutation along the way.

Yet, even without evolutionary pressure, we can understand that due to the randomness of sexual selection, a group starting with $n$ different haplogroups, would eventually converge to only $1$.

**The question is, how fast will it converge ?**

Monte carlo approach
---
The MC approach, is simply simulating the sexual selection in each generation and looking at the number of different haplogroups in each generation.

We start with $n$ men and $n$ women and in each generation we randomize $n$ sons and $n$ daughters that would be born to random chosen parents.

In [1]:
from random import randint
class haplogroup_mc:
    def __init__(self,n):
        self.n=n
        self.generation=0
        self.men=range(n)
    def reproduce(self):
        self.generation+=1
        new_men=[]
        new_women=[]
        for i in range(self.n):
            father=self.men[randint(0,self.n-1)]
            new_men.append(father)
        self.men=new_men
    def converged(self):
        first=self.men[0]
        for m in self.men:
            if m!=first:
                return False
        return True
    def run_until_convergence(self):
        while not self.converged():
            self.reproduce()
        return self.generation

In [2]:
def num_of_generations_until_convergence(population_size,iteration_count=1000):
    generation_sum=0
    for i in range(iteration_count):
        h=haplogroup_mc(population_size)
        generation_sum+=h.run_until_convergence()
    generation_avg=float(generation_sum)/iteration_count
    return generation_avg

In [3]:
import numpy as np
from sklearn import linear_model
def lin_reg(x,y):
    linreg=linear_model.LinearRegression(fit_intercept=True)
    linreg.fit([[x_] for x_ in x],y)
    slope=linreg.coef_[0]
    intercept=np.mean(x)
    return ("Y={a}*n + {b}".format(a=slope,b=intercept))

The results of the simulation looks linear, running a linear regression yields:
$$y\approx 2.01n$$

In [8]:
import matplotlib.pyplot as plt
x=range(5,50)
y=map(num_of_generations_until_convergence,x)
plt.plot(x,y)
print (lin_reg(x,y))

Y=2.01568669302*n + 27.0


In [10]:
x=range(5,100,10)
y=map(num_of_generations_until_convergence,x)
plt.plot(x,y)
print (lin_reg(x,y))

Y=1.97355393939*n + 50.0


Mathematical approach
---


Given a set $S$ such that $|S|=n$

For $x\in S$ define $A_x$ to be the event that $x\in T$.

Then $|T|=\sum_{x\in S} 1_{A_x}$ which has expected value
$$\mathbb{E}(X)=\sum_{x\in S} \mathbb{P}(A_x)=n\left[1-\left(1-{1\over n}\right)^n\right]\approx n(1-e^{-1})$$
Lets verify this approximation using one generation MC:

In [114]:
from random import randint
from math import e
n=100
mc_iterations=1000
mc_avg=0
for i in xrange(mc_iterations):
    mc_val=len(set([randint(0,n-1) for i in xrange(n)]))
    mc_avg+=mc_val
mc_avg/=float(trials)
expected=n*(1-1/e)
print ((avg_mc,expected))

(63.6, 63.212055882855765)


Number of generations until convergence
---
$$n(1-\frac{1}{e})^g\leq 1$$
$$g\ln(1-\frac{1}{e})\leq \ln\frac{1}{n}$$
$$g\geq \frac{\ln\frac{1}{n}}{\ln(1-\frac{1}{e})}$$
$$g\geq \frac{\ln{n}}{\ln(\frac{e}{e-1})}$$
$$g\geq \frac{\ln{n}}{1-\ln(e-1)}$$

In [3]:
from math import log,e
n=100
g=log(n)/(1-log(e-1))
g

10.040156377127712

The mathematical analysis shows that the number of generation is actually logarithmic !

With $2.18$ as the scale parameter

In [7]:
1/(1-log(e-1))

2.180192256016155